In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta,date
import plotly.plotly as py
import plotly.graph_objs as go
from itertools import product
import plotly.figure_factory as ff

In [2]:
dd = pd.read_csv('duedatediag.csv')
dd = dd.rename( columns={"DOCKNO": "Failed", "TOTAL": "Total"})
dd['Org'] = dd.apply(lambda x: x['specific_pt'].split('-')[0],axis=1)
dd['Dest'] = dd.apply(lambda x: x['specific_pt'].split('-')[1],axis=1)
virtuallocs = ['AMCF','AMDO','BBIB','BDQB','IXGF','BLRF','BWDB','BRGO','CCC','CJBC','DELO','GZBB','HYDO','IDRB','JAIC','JLRB','KNB','LKOB','MAAC','NAGB','PLGB','PNQO','PNQK','RPRB','SMBF','SNRB','VPIB','VGAF','SLMF','BHOB']
dd['Reason'] = dd.apply(lambda x: 'Virtual' if x['Org'] in virtuallocs or x['Dest'] in virtuallocs else x['Reason'], axis=1)
dd['Date'] = dd.apply(lambda x: datetime.strptime(x['Timestamp'],"%d/%m/%y"),axis=1)

In [3]:
loclist = list(dd['Org'].unique())+list(dd['Dest'].unique())
###colormapping to be done here

In [4]:
def printfigs(df,reason,threshold,startdate=None,enddate=None):
    if not startdate:
        startdate = min(dd['Date'])
    else:
        startdate = datetime.strptime(startdate,'%d-%m-%Y')
    if not enddate:
        enddate = max(dd['Date'])
    else:
        enddate = datetime.strptime(enddate,'%d-%m-%Y')
    df = df[(df['Date']>=startdate)&(df['Date']<=enddate)]
    data = []
    
    df = df[df['Reason']==reason]
    pivot = df.pivot_table(index = ['specific_pt','Org'], values = ['Total','Failed'], aggfunc=np.sum).reset_index()
    pivot['Perc'] = pivot.apply(lambda x: np.round(x['Failed']*100.0/x['Total'],2),axis=1)
    finalperc = np.round(pivot['Failed'].sum()*100.0/pivot['Total'].sum(),2)
    total = pivot['Total'].sum()
    pivot = pivot[pivot['Total']>=threshold]
    for counter, org in enumerate(pivot['Org'].unique()):
        if reason == 'Virtual' and org in virtuallocs:
            markerdata = dict(size = 10,color = 'rgba(255, 182, 193, .9)')
        elif reason == 'Virtual':
            markerdata = dict(size = 10,color = 'rgba(27, 224, 68, 0.7)')
        else:
            markerdata = dict(size = 10) ### map the regioncode here later
        
            
        trace = go.Scatter(
            x = pivot[pivot['Org']==org]['Failed'],
            y = pivot[pivot['Org']==org]['Perc'],
            name = org,
            mode = 'markers',
            marker = markerdata,
            ### the color should be linked to the org's region for location
            text= pivot[pivot['Org']==org]['specific_pt']
            )
        data.append(trace)


    layout = dict(title = '{0} failure {1}% out of {2} cons'.format(reason,np.round(finalperc,0),total),
                  yaxis = dict(zeroline = False),
                  xaxis = dict(zeroline = False)
                 )

    fig = dict(data=data, layout=layout)
    return fig

In [5]:
py.iplot(printfigs(dd,'Location',17,'01-08-2017'))